In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPClassifier
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,f1_score

C:\Users\JUHI\Anaconda2\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
customer=pd.read_excel('LU_CUSTOMER.xlsx')
item=pd.read_excel('LU_ITEM.xlsx')
order=pd.read_excel('LU_ORDER.xlsx')
detail=pd.read_excel('ORDER_DETAIL.xlsx')
customer.head()

,CUSTOMER_ID,CUST_LAST_NAME,CUST_FIRST_NAME,CUST_BIRTHDATE,AGE,EMAIL,ADDRESS,ZIPCODE,INCOME_ID,BRACKET_DESC,CUST_CITY_ID,CUST_CITY_NAME
0,1,Aaronson,Maxwell,1961-07-11,55,maaronson93@aol.demo,9865 Marion Place Apt.,7071,6,61-70K,221,Lindhurst
1,2,Abarca,Hugh,1934-09-17,82,habarca60@hotmail.demo,1660 Park Ave.,53233,3,31-40K,251,Milwaukee
2,3,Abelson,Hazel,1943-04-08,74,habelson98@hotmail.demo,1882 St. Johns,22182,3,31-40K,400,Vienna
3,4,Abern,Brooks,1937-12-05,79,babern40@hotmail.demo,245 Eastwood,60949,3,31-40K,229,Ludlow
4,5,Abram,Ross,1939-02-28,78,rabram59@hotmail.demo,45 South Deer Creek Drive,52537,3,31-40K,37,Bloomfield


In [18]:
item.head()

,ITEM_ID,ITEM_NAME,ITEM_LONG_DESC,DISC_CD,UNIT_PRICE,UNIT_COST,SUBCAT_ID,SUPPLIER_ID,BRAND_ID
0,90,Self-Management and Efficiency,Self-Management and Efficiency: Large Corporat...,12090,25,19,12,109,174
1,91,Harman Kardon Digital Surround Sound Receiver,Digital Surround Sound Receiver,21091,1000,739,21,202,203
2,92,Harman Kardon AM/FM Stereo Receiver,HK3370 AM/FM Stereo Receiver,21092,500,373,21,202,203
3,93,Harman Kardon Dolby Digital Receiver,AVR45 Dolby Digital Receiver,21093,700,518,21,202,203
4,94,GPX Portable CD Player with Bass Boost,Portable CD Player with Bass Boost,21094,30,22,21,203,202


In [17]:
detail.head()



,ORDER_ID,CUSTOMER_ID,ITEM_ID,ORDER_DATE,EMP_ID,PROMOTION_ID,QTY_SOLD,UNIT_PRICE,UNIT_COST,DISCOUNT
0,208,363,275,2003-01-08,5,1,1,19,16,0
1,209,375,218,2003-01-29,8,1,1,17,13,0
2,209,375,219,2003-01-29,8,1,1,15,11,0
3,210,228,185,2003-01-09,4,1,1,9,7,0
4,211,325,147,2003-01-12,12,1,1,90,67,0


In [5]:
order.head()

,ORDER_ID,CUSTOMER_ID,PYMT_TYPE,SHIPPER_ID
0,1,26,2,2
1,2,59,1,1
2,3,101,2,2
3,4,66,3,2
4,5,180,4,1


In [6]:
item_detail=pd.merge(item, detail, on=['ITEM_ID'])

In [21]:
cid=pd.merge(item_detail,customer, on=['CUSTOMER_ID'])

In [22]:
df=pd.merge(cid,order,on=['CUSTOMER_ID','ORDER_ID'])

In [20]:
df.head()

,ITEM_ID,DISC_CD,UNIT_PRICE_x,UNIT_COST_x,ORDER_ID,CUSTOMER_ID,ORDER_DATE,EMP_ID,PROMOTION_ID,QTY_SOLD,UNIT_PRICE_y,UNIT_COST_y,DISCOUNT,AGE,INCOME_ID,BRACKET_DESC,PYMT_TYPE,city
0,90,12090,25,19,446,449,2003-01-21,21,1,1,25,19,0,65,2,21-30K,3,12
1,247,35247,19,14,446,449,2003-01-21,21,1,1,19,14,0,65,2,21-30K,3,12
2,104,21104,600,445,134223,449,2003-11-18,8,1,1,600,445,0,65,2,21-30K,3,12
3,178,26178,400,296,134223,449,2003-11-18,8,1,1,400,296,0,65,2,21-30K,3,12
4,324,44324,15,14,134223,449,2003-11-18,8,1,1,15,14,0,65,2,21-30K,3,12


In [10]:
df.dropna(inplace=True)
df=df.drop(['ITEM_NAME', 'ITEM_LONG_DESC','SUBCAT_ID','SUPPLIER_ID','BRAND_ID','CUST_LAST_NAME','CUST_FIRST_NAME','CUST_BIRTHDATE','EMAIL','CUST_CITY_ID','ADDRESS','ZIPCODE','CUST_CITY_ID','SHIPPER_ID'],axis=1)

In [14]:
df.head()

,ITEM_ID,DISC_CD,UNIT_PRICE_x,UNIT_COST_x,ORDER_ID,CUSTOMER_ID,ORDER_DATE,EMP_ID,PROMOTION_ID,QTY_SOLD,UNIT_PRICE_y,UNIT_COST_y,DISCOUNT,AGE,INCOME_ID,BRACKET_DESC,PYMT_TYPE,city
0,90,12090,25,19,446,449,2003-01-21,21,1,1,25,19,0,65,2,21-30K,3,12
1,247,35247,19,14,446,449,2003-01-21,21,1,1,19,14,0,65,2,21-30K,3,12
2,104,21104,600,445,134223,449,2003-11-18,8,1,1,600,445,0,65,2,21-30K,3,12
3,178,26178,400,296,134223,449,2003-11-18,8,1,1,400,296,0,65,2,21-30K,3,12
4,324,44324,15,14,134223,449,2003-11-18,8,1,1,15,14,0,65,2,21-30K,3,12


In [13]:
le=LabelEncoder()
df['city']=le.fit_transform(df['CUST_CITY_NAME'])
df=df.drop(['CUST_CITY_NAME'],axis=1)
